In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets
import matplotlib.pyplot as plt
%matplotlib inline
device = "cuda:0"

In [3]:
train_dir = "/content/drive/MyDrive/melanoma_cancer_dataset/train"
test_dir = "/content/drive/MyDrive/melanoma_cancer_dataset/test"
# Pytorch ImageFolder and DataLoader are way too overpowered.
train_data = torchvision.datasets.ImageFolder(train_dir, transform=torchvision.transforms.ToTensor())
test_data = torchvision.datasets.ImageFolder(test_dir, transform=torchvision.transforms.ToTensor())
len(train_data)
#9605 training images
len(test_data)
#1000 testing images

1000

In [4]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
print(train_data[0][0].shape)
# sanity checking, imagefolder changes tensor shape but thats alright I guess.
for x, t in train_loader:
    print(x, t)
    print(x.shape)
    break

torch.Size([3, 300, 300])
tensor([[[[0.4471, 0.4510, 0.4588,  ..., 0.3176, 0.3059, 0.2980],
          [0.4510, 0.4549, 0.4627,  ..., 0.3176, 0.3059, 0.2980],
          [0.4627, 0.4627, 0.4667,  ..., 0.3294, 0.3176, 0.3098],
          ...,
          [0.5490, 0.5529, 0.5569,  ..., 0.3922, 0.3765, 0.3686],
          [0.5490, 0.5529, 0.5569,  ..., 0.3843, 0.3686, 0.3608],
          [0.5451, 0.5490, 0.5569,  ..., 0.3804, 0.3647, 0.3569]],

         [[0.3294, 0.3333, 0.3412,  ..., 0.2471, 0.2353, 0.2275],
          [0.3333, 0.3373, 0.3451,  ..., 0.2471, 0.2353, 0.2275],
          [0.3451, 0.3451, 0.3490,  ..., 0.2471, 0.2353, 0.2275],
          ...,
          [0.4392, 0.4431, 0.4471,  ..., 0.3098, 0.3059, 0.2980],
          [0.4392, 0.4431, 0.4471,  ..., 0.3020, 0.2980, 0.2902],
          [0.4353, 0.4392, 0.4471,  ..., 0.2980, 0.2941, 0.2863]],

         [[0.3294, 0.3333, 0.3412,  ..., 0.3020, 0.2902, 0.2824],
          [0.3333, 0.3373, 0.3451,  ..., 0.3020, 0.2902, 0.2824],
          [0.345

In [21]:
class MelanomaCNN(nn.Module):
    def __init__(self, width=3, normalize_batch=True):
        """
        Init method for the model.
        """
        super(MelanomaCNN, self).__init__()
        self.width = width
        self.bn = normalize_batch
        self.maxpool_layer = nn.MaxPool2d(3,3)
        self.maxpool_layer.cuda()
        # first conv2d layer
        self.conv1 = nn.Conv2d(
            in_channels=self.width,
            out_channels=16,
            kernel_size=7,
        )
        self.conv1.cuda()

        self.conv2 = nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=7
        )
        self.conv2.cuda()

        self.conv3 = nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=3
        )
        self.conv3.cuda()

        self.conv4 = nn.Conv2d(
            in_channels=64,
            out_channels=128,
            kernel_size=3
        )
        self.conv4.cuda()
        # 3 batch norm channels corresponding to output num channels of each conv
        # layer.
        if self.bn:
          #self.bn1 = nn.BatchNorm2d(self.width)
          self.bn2 = nn.BatchNorm2d(32)
          self.bn2.cuda()
          #self.bn3 = nn.BatchNorm2d(64)
          #self.bn4 = nn.BatchNorm2d(128)

        self.fc1 = nn.Linear(512 ,256)
        self.fc1.cuda()
        self.fc2 = nn.Linear(256,2)
        self.fc2.cuda()

        # final output should be batch size * 2

    def forward(self,x):
        x = self.maxpool_layer(torch.relu(self.conv1(x)))
        x.cuda()
        #if self.bn:
            #x = self.bn1(x)

        x = self.maxpool_layer(torch.relu(self.conv2(x)))
        x.cuda()

        if self.bn:
            x = self.bn2(x)
            x.cuda()

        x = self.maxpool_layer(torch.relu(self.conv3(x)))
        x.cuda()

        #if self.bn:
            #x = self.bn3(x)

        x = self.maxpool_layer(torch.relu(self.conv4(x)))
        x.cuda()

        #if self.bn:
          #x = self.bn4(x)

        # flatten the image.
        x = x.view(x.shape[0],-1)
        x.cuda()

        # might swap relu out for smth else
        x = torch.relu(self.fc1(x))
        x.cuda()
        return self.fc2(x)

In [19]:
def get_accuracy(model, data, device="cuda:0"):
    """
    Accuracy function, self explanatory.
    """
    loader = torch.utils.data.DataLoader(data, batch_size=32)
    model.to(device)
    model.eval() # annotate model for evaluation (important for batch normalization)
    correct = 0
    total = 0
    for imgs, labels in loader:
        labels = labels.to(device)
        output = model(imgs.to(device))
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

def train_model(model,
                train_data,
                valid_data,
                batch_size=32,
                weight_decay=0.0,
                learning_rate=0.075,
                num_epochs=10,
                plot_every=20,
                plot=True,
                device=torch.device("cuda:0")):
    train_loader = torch.utils.data.DataLoader(train_data,
                                               batch_size=batch_size,
                                               shuffle=True)
    model = model.to(device) # move model to GPU if applicable
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(),
                           lr=learning_rate,
                           weight_decay=weight_decay)
    # for plotting
    iters, train_loss, train_acc, val_acc = [], [], [], []
    iter_count = 0 # count the number of iterations that has passed

    try:
        for epoch in range(num_epochs):
            for imgs, labels in iter(train_loader):
                if imgs.size()[0] < batch_size:
                    continue
                labels = labels.to(device)
                imgs = imgs.to(device)
                model.train()
                out = model(imgs)
                loss = criterion(out, labels)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

                iter_count += 1
                if iter_count % plot_every == 0:
                    loss = float(loss)
                    tacc = get_accuracy(model, train_data, device)
                    vacc = get_accuracy(model, valid_data, device)
                    print("Iter %d; Loss %f; Train Acc %.3f; Val Acc %.3f" % (iter_count, loss, tacc, vacc))

                    iters.append(iter_count)
                    train_loss.append(loss)
                    train_acc.append(tacc)
                    val_acc.append(vacc)
    finally:
        plt.figure()
        plt.plot(iters[:len(train_loss)], train_loss)
        plt.title("Loss over iterations")
        plt.xlabel("Iterations")
        plt.ylabel("Loss")

        plt.figure()
        plt.plot(iters[:len(train_acc)], train_acc)
        plt.plot(iters[:len(val_acc)], val_acc)
        plt.title("Accuracy over iterations")
        plt.xlabel("Iterations")
        plt.ylabel("Accuracy")
        plt.legend(["Train", "Validation"])

In [23]:
model = MelanomaCNN()